# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - Kociemba's two-phase algo</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [EDA](#3)
* [FUNCTIONS](#4)
* [SOLVE](#5)

Code modified from: https://www.kaggle.com/code/wrrosa/santa-2023-kociemba-s-two-phase-algo-1-116-550

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

!git clone https://github.com/trincaog/magiccube.git

In [1]:
import os
import time


import random
from typing import Dict, List
import zipfile
import sqlite3

import pandas as pd
import numpy as np
from sympy.combinatorics import Permutation


from enum import Enum
from math import sqrt
from ast import literal_eval
from termcolor import colored

import kociemba


In [2]:
database_file = '../solutions.db'
solution_method = "Kociemba two-phase algo"


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [3]:
with zipfile.ZipFile('../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f)        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f)
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

In [4]:
puzzle_info['allowed_moves_count'] = puzzle_info['allowed_moves'].map(lambda x: {k: Permutation(v) for k, v in eval(x).items()})
paths = pd.merge(puzzles, puzzle_info, how='left', on='puzzle_type')
paths = pd.merge(paths, submission, how='left', on='id')

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >EDA<br><div> 

Find BC edge and convert to AB. 7/10, 14/21, 23/30, 32/39, 12/41, 16/46, 34/52, 5/19, 1/28, 3/37, 25/50, 43/48
              7     10      14            21
A;B;A;B;A;B;A;B;A;B;C;B;C;B;C;B;C;B;C;D;C;D;C;D;C;D;C;D;E;D;E;D;E;D;E;D;E;F;E;F;E;F;E;F;E;F;A;F;A;F;A;F;A;F
C;B;D;B;B;C;F;E;A;B;D;D;B;C;A;A;E;B;E;F;A;F;A;B;F;A;F;C;F;A;C;E;A;D;D;C;B;E;C;E;F;D;D;A;E;B;F;E;D;D;C;F;C;E

         Positions                                Solution State

         +---------+                               +---------+
         | 0  1  2 |                               | A  B  A |
         |         |                               |         |
         | 3  4  5 |                               | B  A  B |
         |         |                               |         |
         | 6  7  8 |                               | A  B  A |
+--------+---------+--------+--------+    +--------+---------+--------+--------+
| 36 37 38| 9 10 11|18 19 20|27 28 29|    | E  F  E| B  C  B | C  D  C| D  E  D|
|         |        |        |        |    |        |         |        |        |
| 39 40 41|12 13 14|21 22 23|30 31 32|    | F  E  F| C  B  C | D  C  D| E  D  E|
|         |        |        |        |    |        |         |        |        |
| 42 43 44|15 16 17|24 25 26|33 34 35|    | E  F  E| B  C  B | C  D  C| D  E  D|
+--------+---------+--------+--------+    +--------+---------+--------+--------+
         | 45 46 47|                               | F  A  F |
         |         |                               |         |
         | 48 49 50|                               | A  F  A |
         |         |                               |         |
         | 51 52 53|                               | F  A  F |
         +---------+                               +---------+
         

         Positions                                Solution State

         +---------+                               +---------+
         | 0  1  2 |                               | A  A  A |
         |         |                               |         |
         | 3  4  5 |                               | A  A  A |
         |         |                               |         |
         | 6  7  8 |                               | A  A  A |
+--------+---------+--------+--------+    +--------+---------+--------+--------+
| 36 37 38| 9 10 11|18 19 20|27 28 29|    | E  E  E| B  B  B| C  C  C| D  D  D|
|         |        |        |        |    |        |        |        |        |
| 39 40 41|12 13 14|21 22 23|30 31 32|    | E  E  E| B  B  B| C  C  C| D  D  D|
|         |        |        |        |    |        |        |        |        |
| 42 43 44|15 16 17|24 25 26|33 34 35|    | E  E  E| B  B  B| C  C  C| D  D  D|
+--------+---------+--------+--------+    +--------+---------+--------+--------+
         | 45 46 47|                               | F  F  F |
         |         |                               |         |
         | 48 49 50|                               | F  F  F |
         |         |                               |         |
         | 51 52 53|                               | F  F  F |
         +---------+                               +---------+



<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [5]:
translations = {
    'corners': {
        "N8N11N18": "ABC", 
        "N2N20N27": "ACD",
        "N0N29N36": "ADE",
        "N6N9N38": "ABE",
        "N15N44N45": "BEF", 
        "N17N24N47": "BCF",
        "N26N33N53": "CDF",
        "N35N42N51": "DEF"
    },
    'faces': {
        'N4':  'A',
        'N13': 'B',
        'N22': 'C',
        'N31': 'D',
        'N40': 'E',
        'N49': 'F',
    }
}

In [6]:
def relabel_3x3x3(id, state):
    dim = 3
    faces = list(range(4, 6*dim**2, dim**2))
    edges = [(7, 10), (14, 21), (23, 30), (32, 39), (12, 41), (16, 46), (34, 52), (5, 19), (1, 28), (3, 37), (25, 50), (43, 48)]
    corners = [(8, 11, 18), (2, 20, 27), (0, 29, 36), (6, 9 , 38), (15, 44, 45), (17, 24, 47), (26, 33, 53), (35, 42, 51)]
    # Split the string into individual elements.
    state_list = state.split(';')
    if id in range(140, 150):
        # For each element, determine the replacement based on the criteria.
        for i, val in enumerate(state_list):
            # Extract the number part from the string like 'N33' -> 33.
            num = int(val[1:])
            # Replace based on the given criteria.
            if 0 <= num <= 8:
                state_list[i] = 'A'
            elif 9 <= num <= 17:
                state_list[i] = 'B'
            elif 18 <= num <= 26:
                state_list[i] = 'C'
            elif 27 <= num <= 35:
                state_list[i] = 'D'
            elif 36 <= num <= 44:
                state_list[i] = 'E'
            elif 45 <= num <= 53:
                state_list[i] = 'F'

    elif id in range(130, 140):
        translations = {
            "AC": "FB",
            "AD": "FC",
            "AE": "FD",
            "AF": "FE",
    
            "BC": "AB",
            "BD": "AC",
            "BF": "AE",
            "BE": "AD",
    
            "CD": "BC",
            "CF": "BE",
            "DE": "CD",
            "EF": "DE",
            
        }
        
        for e1, e2 in edges:
            c1 = state_list[e1]
            c2 = state_list[e2]
            # Sort the strings
            sorted_strings = sorted([c1, c2])
            # Concatenate them
            concatenated_string = ''.join(sorted_strings)
            t1, t2 = translations[concatenated_string]
            if c1 == sorted_strings[0]:
                state_list[e1] = t1
                state_list[e2] = t2
            else:
                state_list[e1] = t2
                state_list[e2] = t1
    return ';'.join(state_list)            
        

In [7]:
U = ['U', 'F', 'R', 'B', 'L', 'D']

def state2ubl(state):

    dct = {}
    for u in range(len(U)):
        dct[state.split(';')[4+u*9]] = U[u]

    s = ''.join([dct[f] for f in state.split(';')])
    
    return s[:9] + s[18:27] + s[9:18] + s[45:] + s[36:45] + s[27:36]

In [8]:
moves = eval(puzzle_info.loc[puzzle_info['puzzle_type'] == 'cube_3/3/3']['allowed_moves'].values[0])
for move in list(moves):
    moves['-'+move] = np.argsort(moves[move]).tolist()

M = {}
M["U"] = '-d2'
M["R"] = "r0"
M["B"] = "-f2"
M["F"] = "f0"
M["L"] = "-r2"
M["D"] = "d0"
for m in list(M):
    M[m+"2"] = M[m] + '.' + M[m]
    if "-" in M[m]:
        M[m+"'"] = M[m].replace("-","")
    else:
        M[m+"'"] = "-"+M[m]

NameError: name 'state' is not defined

<a id="5"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [11]:
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

for i in paths.loc[paths['puzzle_type'] == 'cube_4/4/4'].iterrows():
    id = i[1]['id']
    # try:
        
    cur_state = i[1]['initial_state']
    sol_state = i[1]['solution_state']
    print(state2ubl(cur_state))
    break
    if id >= 130 and id <= 149:
        cur_state = relabel_3x3x3(id, cur_state)
        sol_state = relabel_3x3x3(id, sol_state)
        
    sol = kociemba.solve(state2ubl(cur_state))

    new_state = cur_state
    mmoves = '.'.join([M[m] for m in sol.split(' ')])

    for move in mmoves.split('.'):
        new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
    I = ['r0.r1.r2','d0.d1.d2','f0.f1.f2']
    for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
        temp_state = new_state
        if len(init_moves) > 0:
            for move in init_moves.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))

        if temp_state == sol_state:
            print(f'solved id: {id}')
            if len(init_moves) > 0:
                mmoves += '.' + init_moves
                
            mmoves_length = len(mmoves.split('.'))
            if len(paths.iloc[id,7].split('.')) > mmoves_length:
                print(f"improved: new length {mmoves_length} vs current length {len(paths.iloc[id,7].split('.'))}")
                paths.iloc[id,7] = mmoves
                
                select_query = "SELECT count FROM solutions WHERE id = ?"
    
                # Execute the query
                cursor.execute(select_query, (id,))
                response = cursor.fetchone()
                best_moves_length = response[0]
                
                if mmoves_length < best_moves_length:
                    # Insert the moves into the database
                    insert_query = ("INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)")
                    cursor.execute(insert_query, (id, mmoves, mmoves_length, solution_method))
                    conn.commit()
            break
    # except:
    #     print(f'Error with id {id}')
    
    
# Commit the changes and close the connection
conn.commit()
conn.close()

# paths[['id','moves']]

KeyError: 'A'

In [10]:
i[1]['solution_state']

'N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;N14;N15;N16;N17;N18;N19;N20;N21;N22;N23;N24;N25;N26;N27;N28;N29;N30;N31;N32;N33;N34;N35;N36;N37;N38;N39;N40;N41;N42;N43;N44;N45;N46;N47;N48;N49;N50;N51;N52;N53'

In [14]:
sol_state

'A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F'

In [15]:
cur_state


'B;A;A;A;C;B;C;F;A;C;A;D;F;D;E;A;D;C;A;B;F;A;B;C;A;B;F;C;C;D;A;F;C;F;E;C;A;A;E;C;B;E;E;B;C;A;D;E;D;D;B;E;B;D;A;D;E;F;C;B;A;C;B;D;F;F;D;E;E;F;D;C;F;D;C;E;A;F;B;B;F;E;C;D;F;B;E;B;E;D;F;F;E;E;D;B'